# Language Model and RNN


## Table of Content
1. N-gram LM
2. Perplexity
3. RNNLM


Complete the following codes to accomplish tasks of **N-gram Language model (LM)** and **RNN Language Model (RNN-LM)**. We will use the following book as text corpus

*   Inputs: book *The Adventures Of Tom Sawyer*

> The book's plain text utf-8 file can be downloaded from Project Gutenberg (https://www.gutenberg.org/). Please rename and place it at "/content/drive/MyDrive/SMU_MITB_NLP/week4/the_adventures_of_tom_sawyer.txt".

Below we have provides some basic codes to prepare your data or your model, but you are free to use your own.

Or, you may choose a larger corpus: [Works_of_William_Shakespeare](https://www.gutenberg.org/ebooks/100)

It is also recommended to request a **GPU** for the RNN-LM training.

In [47]:
#@title show your CPU or GPU details
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 155467119106591047
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14626652160
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14378820008859485104
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [48]:
#@title connect google drive folder

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_NLP/week4/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SMU_MITB_NLP/week4


In [49]:
#@title import useful packages
import gensim
import pandas as pd
import smart_open
from nltk import bigrams
import math
import pickle

import re, collections
from collections import defaultdict, Counter
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)


In [50]:
#@title Utility functions for reading and preprocessing data

def read_corpus(file_name):
    data = []
    new_line = ''
    with open(file_name, "r", encoding="utf-8") as fin:
        for line in fin:
            if line not in ['\n','\r\n']:
                new_line += line.strip() + ' '
                continue
            tokens = preprocess(new_line)
            if len(tokens) < 5: continue    # filter out short sentence
            yield tokens
            new_line = ''

def preprocess(sent):
    return gensim.utils.simple_preprocess(sent, min_len=1)


In [51]:
# clean_data = list(read_corpus("./Works_of_William_Shakespeare.txt"))
clean_data = list(read_corpus("./the_adventures_of_tom_sawyer.txt"))
print("The number of sentences in the book: ", len(clean_data))
print(clean_data[:2])

The number of sentences in the book:  1828
[['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'tom', 'sawyer', 'complete', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www', 'gutenberg', 'org', 'if', 'you', 'are', 'not', 'located', 'in', 'the', 'united', 'states', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'ebook'], ['title', 'the', 'adventures', 'of', 'tom', 'sawyer', 'complete']]


## N-gram language model

N-gram language model aims to collect statistics about how frequent different n-grams are, a.k.a, a chunk of n consecutive words.

Such that, we are able to:

1.   either assign a joint probability for a sequence of words;
2.   or, predict upcoming word given a couple of word.

Given a text corpus, the key of training a n-gram language model is to count every unique n-gram, and transform them into probabilities by:

$p(w_i|w_{i-n-1:i-1})=\frac{count(w_{i-n-1:i})}{count(w_{i-n-1:i-1})}$

where $w_{i-n-1:i}$ denotes a n-gram from the $(i-n-1)$th word to the $i$th word.


### Markov Assumption

Markov Assumption provides theory fundation to simplify n-gram LM, which usually suffer from sparsity issue.

It assumes that the future state is indepedent of the past states conditioned on the current states. That is, the probabilities of upcoming words depend on only the last k words, instead of all preceding words in the given sequence:

$p(w_i|w_{i-n-1:i-1})\approx p(w_i|w_{i-k:i-1})$

### Bi-gram LM training

Now, let's assume $n=2$ or $k=1$, and finish a bi-gram LM training with **Add-k** smoothing!

You need to complete the following codes of function named *bigram_lm*, which is able to:

1.   input: *data* is a list of sentence, where each sentence is a list of tokens (i.e., already splitted).
2. Input: *smooth_k* is a hyperparameter for smoothing, 1 by default.
3. output: *count_bigram_model* is a python dictionary or collections.defaultdict, which has **two** keys (e.g., each value of the key is also a dictionary) and the values should be the smoothed count number of the bigram.
4. *prob_bigram_model* is similar with *count_bigram_model*, but the value is the probabilities $p(w_i|w_{i-1})$ for each bigram in the book;


**hint**: you may use the following 3rd party package:

*   nltk.bigrams


In [52]:
#@title bigram language model
def bigram_lm(data, smooth_k=1):
    # Create a placeholder for model
    count_bigram_model = defaultdict(lambda: defaultdict(lambda: 0))
    prob_bigram_model = defaultdict(lambda: defaultdict(lambda: 0))

    # Count frequency of co-occurance and transform the counts to probabilities
    # do not change the code above
    # write your code here

    # Count frequency of co-occurrence
    unigram_counts = defaultdict(lambda: 0)
    for sentence in data:
        for bi in bigrams(sentence, pad_right=True, pad_left=True):
            count_bigram_model[bi[0]][bi[1]] += 1
            unigram_counts[bi[0]] += 1
    # Calculate total vocabulary size for smoothing
    vocabulary_size = len(set(word for sentence in data for word in sentence))

    # Transform the counts to probabilities with Add-k smoothing
    for prev_word in count_bigram_model:
        total_prev_word_count = unigram_counts[prev_word] + (vocabulary_size * smooth_k)
        for word in count_bigram_model[prev_word]:
            # Apply smoothing to the count
            smoothed_count = count_bigram_model[prev_word][word] + smooth_k
            count_bigram_model[prev_word][word] = smoothed_count
            # Calculate probability
            prob_bigram_model[prev_word][word] = smoothed_count / total_prev_word_count

    # do not change the code below

    return count_bigram_model, prob_bigram_model

In [53]:
#@title training bigram language model!

count_bigram_model, prob_bigram_model = bigram_lm(clean_data, 1)

### Word Prediction - Bigram

Congratulations! You have finish the training of Bi-gram LM!

Now, let us test your bigram_model by predicting the top 10 possible words given a word "i" (lower case of "I")!

You may use the following or your own codes for prediction!

In [54]:
#@title inference bigram language model!

def bigram_predict(bigram_model, token):
    result = sorted(dict(bigram_model[token.lower()]).items(), key=lambda x:x[1], reverse=True)
    df = pd.DataFrame(result, columns=['word', 'prob'])
    df['order'] = [i + 1 for i in range(len(result))]
    df = df[['order', 'word', 'prob']]
    print(df.head(10))

In [55]:
bigram_predict(prob_bigram_model, 'i')

   order    word      prob
0      1      ll  0.011834
1      2     don  0.008238
2      3  reckon  0.007541
3      4       m  0.005337
4      5      ve  0.005105
5      6       d  0.005105
6      7     can  0.003945
7      8     was  0.003713
8      9    know  0.003481
9     10   never  0.003249


## Perplexity

Perplexity is a metric to evaluate a Language Model. If the trained LM can assign a higher probability to frequent language and assign a lower probability to infrequent language. Note that lower perplexity denotes a better LM!

Perplexity is defined as:

$PP(s)=p(w_1,\cdots,w_i,\cdots,w_n)^{-\frac{1}{n}}=\sqrt[n]{\prod_{i=1}^{n}\frac{1}{p(w_i|w_{i-n-1:i-1})}}$

Now, let us compute perplexity for the following sentence to evaluate your bi-gram LM!

*   "Wait! I want to tell something."
*   "remember to sumbit your assignment"

**hint**: you may use Add-k smoothing to address sparsity issue (e.g., zero-value count)

In [56]:
#@title compute perplexity

def perplexity(prob_bigram_model, count_bigram_model, sent, smooth_k=1.0):
    sent = preprocess(sent)
    pp = 1.0
    # do not change the code above
    # write your code here
    vocabulary_size = len(set(word for subdict in count_bigram_model.values() for word in subdict))

    for i in range(1, len(sent)):  # Starting from 1 because we need a previous word
        prev_word = sent[i-1]
        current_word = sent[i]

        # Check if there's a previous word model, and it has the current word
        if prev_word in prob_bigram_model and current_word in prob_bigram_model[prev_word]:
            probability = prob_bigram_model[prev_word][current_word]
        else:
            # Apply smoothing if the bigram does not exist
            if prev_word in count_bigram_model:
                total_prev_word_count = sum(count_bigram_model[prev_word].values()) + (smooth_k * vocabulary_size)
            else:
                total_prev_word_count = smooth_k * vocabulary_size
            probability = smooth_k / total_prev_word_count

        # Accumulate the inverse probabilities
        pp *= probability



    # do not change the code below
    return math.pow(1/pp, 1/float(len(sent)))

In [57]:
print(perplexity(prob_bigram_model, count_bigram_model, "To be, or not to be, that is the question:", 1.0))
print(perplexity(prob_bigram_model, count_bigram_model, "remember to sumbit your assignment", 1.0))

516.8196198458153
1350.353217104285


## RNN-LM

Now, we provide an example RNN for your reference, and you are to complete the code for LSTM and GRU.

### RNN

RNN is a kind of neural network that can recurrently take any length of sequential data. It can be regarded as stacked neural network over time.

To build a neural LM, we can use rnn to encode each word in the sequence at each timestamp, the hidden states of current word will take both current word and the hidden states of the preceding word as inputs:

$h^t=\sigma(W_ex^t+W_hh^{t-1})$

where $x^t \in \mathbb{R}^{|V|}$ is the one-hot encoding of word $w^t$, and $W_e \in \mathbb{R}^{d_e \times |V|}$ is the word embedding matrix, $d_e$ is the word embedding dimension, $h^t, h^{t-1}\in \mathbb{R}^{d_h}$ is hidden states at $t$ and $t-1$ timestamps, and $W_h \in \mathbb{R}^{d_h \times d_h}$ is learnable parameters. Normally, we set $h^0$ as initial zero-value hidden state vector.

Note that we have to make sure the same dimension of $d_e$ and $d_h$; otherwise, we may need another linear projection for $W_ex^t$:

$h^t=\sigma(W_{he}W_ex^t+W_hh^{t-1})$

where $W_{he}\in \mathbb{R}^{d_h \times d_e}$ are learnable parameters.

Such that, we are able to use either the hidden states of each timestamp $h^t$ or the final hidden states $h^T$ to make prediction!

$\hat{y}^t=softmax(Uh^t)$

where $U \in \mathbb{R}^{|V| \times d_h}$ is learnable parameters.

*Actually, I ignore the transpose of the above matrix for simplicity!*


### RNN for language model

Now, we regard a sequence of $T$ words as word at different timestamps:

$<w_1,\cdots,w_t,\cdots,w_T>$

and, use RNN to predict each upcoming word given all preceding words:

$p(w_2|w_1),p(w_3|w_{1:2}),\cdots,p(w_T|w_{1:T-1})$

For example, given a sentence "remember to sumbit your assignment"

*   inputs of RNN: a list of tokens 'remember' 'to' 'sumbit' 'your' 'assignment'
*   outputs of RNN: a list of tokens 'to' 'sumbit' 'your' 'assignment' '<SEP>'

where '<SEP>' denotes the end of the sequence.


### using pytorch to implement VanillaRNNLM

Based on Pytorch, we can inherit from *torch.nn.Module* to build your own neural network. More details please refer to the pytorch documentation.

Similar to the example codes for skipgram, you have to finish the functions of *\_\_init\_\_()* and *forward()*, additional helper functions are also ok. The inputs for *\_\_init\_\_* includes:

*   vocab_size
*   hidden_size: here we assume the dimension of hidden states is the same with word embedding size for simiplicity.

For *forward*:
*   input: a tensor with dimension of batch_size\*sequence_length. Note that, in each batch, you need to make sure all sentences have the same length, either by pading enough special tokens to the end of each sentence, or cutting off the overlength sentence.
*   output: a tensor with dimension of batch_size\*sequence_length\*vocab_size. The probability distribution over the vocabulary for each token in the sequence in the batch.

**hint**: we usually replace each token in a sequence with their id in a pre-built vocabulary!

In [58]:
#@title vanilla RNN LM

class VanillaRNNLM(nn.Module):

    # add your codes here
    def __init__(self, vocab_size, hidden_size):
        super(VanillaRNNLM, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size

        self.emb = nn.Embedding(vocab_size, hidden_size, padding_idx=0) # W_e
        self.h2h = nn.Linear(hidden_size, hidden_size)                  # W_h
        self.h2y = nn.Linear(hidden_size, vocab_size)                   # U
        self.softmax = nn.LogSoftmax(dim=1)


    def forward(self, inputs):

        bs, seq_sz = inputs.size()
        hidden = self.init_hidden(bs).to(inputs.device)
        probs = self.init_output_prob(bs, seq_sz).to(inputs.device)

        cur_h = self.emb(inputs)

        for t in range(seq_sz):
            cur_ht = cur_h[:,t,:]
            prev_h = self.h2h(hidden)
            hidden = torch.tanh(cur_ht + prev_h)
            logits = self.h2y(hidden)
            probs[:,t,:] = self.softmax(logits)
        return probs

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

    def init_output_prob(self, batch_size, sequence_length):
        return torch.zeros(batch_size, sequence_length, self.vocab_size)

### RNN Variants - LSTM

LSTM is a special kind of RNN to capture long-term dependency. It bring "add" operation via Gated operation to avoid gradient vanishing in RNN due to the continued product of the derivative of the non-linear activation function, which is usually a very small value.

At time step t, LSTM has two objects: hidden state and cell state. Both are vectors. The information from previous timesteps is to be erased/read/written with three gates: forget gate, input gate, and output gate. The detailed framework is shown as follows:


![](https://drive.google.com/uc?export=view&id=1YSL8h2zRQfR-gf1NpKy7zxQOlo3BbO8h)

The equations for three gates are as follows:

![](https://drive.google.com/uc?export=view&id=1kgXcXTVJiHVtW-j3rhDyc0pAL1wtUwPr)

Thus, LSTM can control the information flow via the following equations:

![](https://drive.google.com/uc?export=view&id=1kxYCc-P1j-swu59XUOsoUElLPl_6ksAh)

### using pytorch to implement VanillaLSTMLM

In [59]:
#@title vanilla LSTM LM

class VanillaLSTMLM(nn.Module):

    # add your codes here
    def __init__(self, vocab_size, hidden_size):
        super(VanillaLSTMLM, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size

        # do not change the code above
        # write your code here

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)

        self.output_layer = nn.Linear(hidden_size, vocab_size)

        self.log_softmax = nn.LogSoftmax(dim=-1)



        # do not change the code below

    def forward(self, inputs):

        bs, seq_sz = inputs.size()

        h_t = self.init_hidden(bs).to(inputs.device)
        c_t = self.init_hidden(bs).to(inputs.device)

        hidden = self.init_hidden_seq(bs, seq_sz).to(inputs.device)
        probs = self.init_output_prob(bs, seq_sz).to(inputs.device)
        # do not change the code above
        # write your code here

        embedded = self.embedding(inputs)

        lstm_out, (h_t, c_t) = self.lstm(embedded, (h_t.unsqueeze(0), c_t.unsqueeze(0)))

        lstm_out = lstm_out.contiguous().view(-1, self.hidden_size)

        logits = self.output_layer(lstm_out)

        log_probs = self.log_softmax(logits)

        probs = log_probs.view(bs, seq_sz, self.vocab_size)


        # do not change the code below
        return probs

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

    def init_hidden_seq(self, batch_size, sequence_length):
        return torch.zeros(batch_size, sequence_length, self.hidden_size)

    def init_output_prob(self, batch_size, sequence_length):
        return torch.zeros(batch_size, sequence_length, self.vocab_size)

### RNN Variants - GRU
GRU improves LSTM by combining both hidden and cell states, as well as coupling gates. Compared with LSTM, GRU is:

*  faster to compute due to less parameters
*  Not necessarily better/worse performance than LSTM
*  Start with LSTM first if you favor long dependencies. Turn to GRU if you favor efficiency.

The detailed framework is shown as follows:


![](https://drive.google.com/uc?export=view&id=1irWKKGelhEUiHtJ92wPkDCv6rGVq-tsH)

The equations for GRU are as follows:

![](https://drive.google.com/uc?export=view&id=11QQSvQByU9q5bREs5QADg-gFQ3-cPYo_)

In [60]:
#@title vanilla GRN LM

class VanillaGRULM(nn.Module):

    # add your codes here
    def __init__(self, vocab_size, hidden_size):
        super(VanillaGRULM, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size

        # do not change the code above
        # write your code here

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

        self.output_layer = nn.Linear(hidden_size, vocab_size)

        self.log_softmax = nn.LogSoftmax(dim=-1)

        self.init_weights()


        # do not change the code below

    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, inputs):
        bs, seq_sz = inputs.size()

        h_t = self.init_hidden(bs).to(inputs.device)

        hidden = self.init_hidden_seq(bs, seq_sz).to(inputs.device)
        probs = self.init_output_prob(bs, seq_sz).to(inputs.device)
        # do not change the code above
        # write your code here

        embedded = self.embedding(inputs)

        gru_out, h_t = self.gru(embedded, h_t.unsqueeze(0))

        gru_out = gru_out.contiguous().view(-1, self.hidden_size)

        logits = self.output_layer(gru_out)

        log_probs = self.log_softmax(logits)

        probs = log_probs.view(bs, seq_sz, self.vocab_size)


        # do not change the code below

        return probs

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

    def init_hidden_seq(self, batch_size, sequence_length):
        return torch.zeros(batch_size, sequence_length, self.hidden_size)

    def init_output_prob(self, batch_size, sequence_length):
        return torch.zeros(batch_size, sequence_length, self.vocab_size)

### Training your RNN-LM

We have provided some codes to prepare the data and train your neural RNN-LM, but it is may not compatible with your codes. So, feel free to use your own codes.

In [61]:
#@title class PreProcessor

# class for preparing data for neural network

class PreProcessor():

    def __init__(self):

        self.model_args = {
            # data preprocessing parameters
            'max_vocabulary_size': 50000,
            'min_occurrence': 5,  # Remove all words that does not appears at least n times.
            'max_seq_length':50,  # make all sentence the same length for batching
            # Training Parameters.
            'hidden_size': 100,
            'learning_rate': 0.01,
            'batch_size': 512,
            'n_epochs': 20,
            'clip': 1,         # clip the gradients in case of too large value
            'init_seed': 42,
            # log parameter
            'verbose': True,  # if output log info
            'log_step': 30,
            'checkpoint_path': "./lm.bin",
        }

        self.w2id_vocab = {'<PAD>':0, '<SEP>':1, '<UNK>':2}
        self.id2w_vocab = {0:'<PAD>', 1:'<SEP>', 2:'<UNK>'}

        self.pad_token_id = self.w2id_vocab['<PAD>']
        self.end_token_id = self.w2id_vocab['<SEP>']
        self.unk_token_id = self.w2id_vocab['<UNK>']

        self.vocab_size = len(self.w2id_vocab)
        self.avg_seq_length = 0

    def save(self, path):
        f = open(path, 'wb')
        pickle.dump(self, f)
        f.close()

    def load(self, path):
        f = open(path, 'rb')
        proc = pickle.load(f)
        f.close()
        return proc

    def set_model_arg(self, key, value):
        self.model_args[key] = value

    def get_model_arg(self, key):
        return self.model_args.get(key, None)

    def get_vocab_size(self):
        self.vocab_size = len(self.w2id_vocab)
        return self.vocab_size

    def get_avg_length(self):
        return self.avg_seq_length

    def get_ids_from_tokens(self, words):
        return [self.w2id_vocab.get(t, self.unk_token_id) for t in words]

    def get_words_from_ids(self, wids):
        return [self.id2w_vocab.get(t, self.id2w_vocab[self.unk_token_id]) for t in wids if t != self.pad_token_id]

    def build_vocab(self, data):
        # Build the dictionary and replace rare words with UNK token.

        flatten_data = sum(data, [])
        count = collections.Counter(flatten_data).most_common(self.get_model_arg('max_vocabulary_size') - 1)
        # Remove samples with less than 'min_occurrence' occurrences.
        if self.get_model_arg('min_occurrence') is not None and self.get_model_arg('min_occurrence') > 0:
            for i in range(len(count) - 1, -1, -1):
                if count[i][1] < self.get_model_arg('min_occurrence'):
                    count.pop(i)
                else:
                    # The collection is ordered, so stop when 'min_occurrence' is reached.
                    break

        num_special_token = self.get_vocab_size()
        for i, (w, _) in enumerate(count):
            wid = i + num_special_token
            self.w2id_vocab[w] = wid
            self.id2w_vocab[wid] = w

        print("Words count:", len(flatten_data))
        print("Unique words:", len(set(flatten_data)))
        print("Vocabulary size:", self.get_vocab_size())


    def convert_examples_to_features(self, examples):
        # process sents: a list of strings
        features = []
        self.avg_seq_length = 0.0
        for (ex_index, example) in enumerate(examples):
            self.avg_seq_length += len(example)
            if len(example)> self.get_model_arg('max_seq_length'):
                example = example[:self.get_model_arg('max_seq_length')-1]
            input_ids, target_ids = self.convert_sentence_to_features(example)


            if ex_index < 5 and self.get_model_arg('verbose'):
                print("*** Example ***")
                print("sent_input_ids: %s" % " ".join([str(x) for x in input_ids]))
                print("sent: {}".format(self.get_words_from_ids(input_ids)))

            features.append((input_ids, target_ids))
        self.avg_seq_length /= len(examples)
        return features

    def convert_sentence_to_features(self, sent):
        # Creating lists that will hold our input and target sequences
        # Remove last token and convert to ids for input sequence
        input_seq = [self.w2id_vocab.get(t, self.unk_token_id) for t in sent]
        # Remove firsts token and convert to ids for target sequence
        target_seq = [self.w2id_vocab.get(t, self.unk_token_id) for t in sent[1:]] + [self.end_token_id]

        input_ids = self.padding_sent(input_seq, max_length=self.get_model_arg('max_seq_length'))
        target_ids = self.padding_sent(target_seq, max_length=self.get_model_arg('max_seq_length'))

        assert len(input_ids) == len(target_ids), "Mismatch with input and target length {} vs {}".format(len(input_ids), len(target_ids))

        return tuple(input_ids), tuple(target_ids)

    # output tuple based ids
    def padding_sent(self, input_ids, max_length=512):
        # Zero-pad up to the sequence length.
        padding_length = max_length - len(input_ids)

        input_ids = input_ids + ([self.pad_token_id] * padding_length)

        assert len(input_ids) == max_length, "Error with input length {} vs {}".format(len(input_ids), max_length)

        return input_ids


    def convert_feature_to_dataset(self, features):

        input_ids = torch.tensor([f[0] for f in features], dtype=torch.long)
        target_ids = torch.tensor([f[1] for f in features], dtype=torch.long)

        return TensorDataset(input_ids, target_ids)

    def get_data_iter(self, features, batch_size, drop_last=True):
        dataset = self.convert_feature_to_dataset(features)

        dataset_sampler = RandomSampler(dataset)    # SequentialSampler(dataset)

        dataloader = DataLoader(dataset, sampler=dataset_sampler, batch_size=batch_size, drop_last=drop_last)

        return iter(dataloader)

In [62]:
#@title utility functions

def check_gpu():
    # torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
    is_cuda = torch.cuda.is_available()

    # If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
    if is_cuda:
        device = torch.device("cuda")
        print("GPU is available")
    else:
        device = torch.device("cpu")
        print("GPU not available, CPU used")
    return device

def set_seed(seed=42):
    # random.seed(seed)
    # np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    return model


In [63]:
#@title preprare dataset
proc = PreProcessor()

# preprocessing data
proc.build_vocab(clean_data)

# save proc
arg_path = "./proc.dat"
proc.save(arg_path)

Words count: 77201
Unique words: 7604
Vocabulary size: 1647


In [64]:
#@title prepare code for training

def train(model, proc, data, device):
    # fetch hyper-parameters
    batch_size = proc.get_model_arg("batch_size")
    n_epochs = proc.get_model_arg('n_epochs')
    learning_rate = proc.get_model_arg("learning_rate")
    verbose = proc.get_model_arg("verbose")
    log_step = proc.get_model_arg("log_step")
    checkpoint_path = proc.get_model_arg("checkpoint_path")
    max_seq_length = proc.get_model_arg("max_seq_length")
    clip = proc.get_model_arg("clip")

    # prepare training dataset
    features = proc.convert_examples_to_features(data)
    data_iter = proc.get_data_iter(features, batch_size)
    if verbose:
        print("avg seq length: {}".format(proc.get_avg_length()))
    # training steps in each epoch
    examples_total_num = len(data)
    max_steps = math.ceil(float(examples_total_num)/batch_size)

    # Define Loss, Optimizer
    criterion = nn.NLLLoss(ignore_index=proc.pad_token_id)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # train!
    for epoch in range(n_epochs):
        total_loss = 0.0
        model.eval()
        for step in range(max_steps):
            model.train()
            optimizer.zero_grad() # Clears existing gradients from previous epoch
            # prepare inputs
            try:
                batch = next(data_iter)
            except StopIteration:
                data_iter = proc.get_data_iter(features, batch_size)
                batch = next(data_iter)
            output = model(batch[0].to(device))
            target_seq = batch[1].to(device)
            loss = criterion(output.view(batch_size*max_seq_length,-1), target_seq.view(-1).long())

            loss.backward() # Does backpropagation and calculates gradients
            # clip gradients to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

            optimizer.step() # Updates the weights accordingly

            total_loss += loss.item()
            if verbose and step%log_step==0:
                print("\rstep: {}/{}, Loss: {:.4f}".format(step, max_steps, loss.item()), end=' ')

        print("epoch: {}/{}, Loss: {:.4f}, saving model to {}".format(epoch, n_epochs, total_loss/max_steps, checkpoint_path))
        save_model(model, checkpoint_path)

In [65]:
#@title Training with saving the model!

model_classes = {'rnn':VanillaRNNLM, 'lstm':VanillaLSTMLM, 'gru':VanillaGRULM}
model_name = 'gru'
# get parameters from preprocessor
init_seed = proc.get_model_arg('init_seed')
hidden_size = proc.get_model_arg('hidden_size')

device = check_gpu()
set_seed(init_seed)

# Instantiate the model with hyperparameters
model_cls = model_classes[model_name]
model = model_cls(vocab_size=proc.get_vocab_size(), hidden_size=hidden_size)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

train(model, proc, clean_data, device)

GPU is available
*** Example ***
sent_input_ids: 3 127 117 577 7 3 578 7 16 239 677 33 577 53 22 3 270 7 1405 743 12 3 412 362 4 212 89 2 7 3 243 28 40 1406 4 18 250 40 2 2 14 337 622 8 209 8 82 49 135 0
sent: ['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'tom', 'sawyer', 'complete', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', '<UNK>', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', '<UNK>', '<UNK>', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're']
*** Example ***
sent_input_ids: 2 3 578 7 16 239 677 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
sent: ['<UNK>', 'the', 'adventures', 'of', 'tom', 'sawyer', 'complete']
*** Example ***
sent_input_ids: 2 1407 2 2 2 2 577 212 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
sent: ['<UNK>', 'mark', '<UNK>', '<UNK>', '<UN

### prepare inference: reload the model!

In [66]:
model_classes = {'rnn':VanillaRNNLM, 'lstm':VanillaLSTMLM, 'gru':VanillaGRULM}
checkpoints = {'rnn':"./lm_rnn.bin", 'lstm':"./lm_lstm.bin", 'gru':"./lm.bin"}
args = {'rnn':"./proc_rnn.dat", 'lstm':"./proc_lstm.dat", 'gru':"./proc.dat"}
model_name = 'gru'

# saved model for seq2seq with or without attention
checkpoint_path = checkpoints[model_name]
arg_path = args[model_name]

proc = PreProcessor()
proc = proc.load(arg_path)
init_seed = proc.get_model_arg('init_seed')
hidden_size = proc.get_model_arg('hidden_size')

device = check_gpu()
set_seed(init_seed)

# Instantiate the model with hyperparameters
model_cls = model_classes[model_name]
model = model_cls(vocab_size=proc.get_vocab_size(), hidden_size=hidden_size)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

model = load_model(model, checkpoint_path)

GPU is available


In [67]:
#@title inference
def greedy_translate(lm_model, proc, out_len, input_sent, device):
    lm_model.eval() # eval mode
    tokens = preprocess(input_sent)
    input_seq = proc.get_ids_from_tokens(tokens)
    out_len -= len(input_seq)

    for t in range(1, out_len):
        input_ids = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0)
        probs = lm_model(input_ids.to(device))
        tokens = probs.argmax(2)
        token = tokens[0][-1].item()
        if (token == proc.end_token_id): break
        input_seq.append(token)

    return ' '.join(proc.get_words_from_ids(input_seq))

In [68]:
greedy_translate(model, proc, 20, "i", device)

'i ll be <UNK>'